to do :
- add th232 --- done! 
- fix formatting -- done!
- check decay chains (weird) 
- add normalization for events divided (how many in flanges)
- check the divide_cryo is done on xp_pri and not on xp 

In [59]:
%run 'init.ipynb'
%run 'geometry_checks.ipynb'
%run 'functions.ipynb'


In [60]:
N = 100000
mass_fv = 40000

materials = [
    "Teflon_TPC",
  #  "SS_OuterCryostat",
  #  "SS_InnerCryostat",
  #  "OuterCryostatReflector",
  #  "SS_BellPlate",
  #  "SS_BellSideWall",
  #  "PmtTpc",
  #  "Copper_TopRing",
  #  "Copper_LowerRing",
  #  "Teflon_Pillar_",
  #  "Copper_FieldGuard_",
  #  "Copper_FieldShaperRing_",
  #  "Electrodes",
  #  "Teflon_BottomTPC",   
  #  "Top_PTFE_Frame",
  #  "Copper_BottomPmtPlate",
  #  "Copper_TopPmtPlate",
    ]



DATE = 'XENONnT_2019-11-12'

isotope_array = ["U238",
                "Co60",
                "K40",
                "Cs137",
                "Th228",
                "U235",
                "Th232",
                "Ra226",
                #"geantinos"
                ]
num_file = '1'

array = np.arange(1, 2)

In [61]:
def calculate_events(data, isotope):
    ss = single_scatter(data)
    fv_ = fv(ss)
    energy_ = energy(fv_, 1, 12)
    en = select_typepri(energy_, isotope)
    events_df = pd.DataFrame()
    events_df = pd.concat([events_df, en])
    events_passing = len(events_df)
    if events_passing ==0:
        en_enlarged = energy(fv_, 1, 100)
        events_passing = (len(en_enlarged)*11)/99
        if events_passing == 0:
            events_passing = 2.3
    return events_passing

In [62]:

def calculate_rate(data, material,isotope,  n_gen, m_fv):
    n_events = calculate_events(data, isotope)
    E = 12
    p = get_parameters(material)
    mass = p['mass']

    contamination = p[isotope]
    err = "err_"+isotope
    err = str(err)
    if (contamination == 0):
        print("|", material, "|", isotope, "|", p[isotope],"+-", p[err],"|","No contamination value", "|" )
        rate = 0
        error_rate = 0
    else:
        time = n_gen / (mass * contamination)
        rate = n_events/(time * m_fv * E)
        alpha = mass / (m_fv * E * n_gen)
        err = "err_"+isotope
        err = str(err)
        error_rate = alpha * np.sqrt((n_events * p[err])**2 + (contamination*np.sqrt(n_events))**2)
        if n_events == 2.3: #it means we set an upper limit
            print("|", material, "|", isotope, "|", p[isotope],"+-", p[err],"| <%2.2f" %rate, "|")
        else:
            print("|", material, "|", isotope, "|", p[isotope],"+-", p[err],"| %2.2f +- %2.2f" %(rate, error_rate), "|")
        return rate, error_rate

In [66]:
def print_results_big_table():    
    for material in materials:
        print("^material ^isotope  ^ contamination ^ rate ^ ")

        for isotope in isotope_array:
                if material == 'Teflon_TPC':
                    rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
                    data = get_xyz(rootfile)
                    mat = "Teflon_TPC_Sliding"
                    calculate_rate(data, mat,isotope, N, mass_fv)
                    mat = "Teflon_TPC_Fixed"
                    calculate_rate(data, mat,isotope, N, mass_fv)

                elif material == "Top_PTFE_Frame":
                    data = get_TopPFTE(isotope)
                    mat = "Top_PTFE_Frame"
                    calculate_rate(data, mat,isotope, N, 4000)

                elif material == "Electrodes":
                    data = get_electrodes(isotope)
                    mat = "SS_Electrodes"
                    calculate_rate(data, mat,isotope, N, 4000)

                elif material == 'SS_OuterCryostat':
                    rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
                    data = get_xyz(rootfile)
                    flange1, flange2, flange3, shell, elongation = divide_outercryo(data)
                    flanges = pd.concat([flange1, flange2, flange3])
                    calculate_rate(flanges, "OuterCryostat_flanges",isotope, N, mass_fv)
                    calculate_rate(shell, "OuterCryostatShell",isotope, N, mass_fv)
                    calculate_rate(elongation, "OuterCryostat_Elongation",isotope, N, mass_fv)

                elif material == 'SS_InnerCryostat':
                    rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
                    data = get_xyz(rootfile)
                    flange1_in, flange2_in, flange3_in, shell_in, elongation_in = divide_innercryo(data)
                    flange_in = pd.concat([flange1_in, flange2_in, flange3_in])
                    calculate_rate(flange_in, "SS_InnerCryostatFlanges",isotope,N, mass_fv)
                    calculate_rate(shell_in, "SS_InnerCryostatShell",isotope,N, mass_fv)
                    calculate_rate(elongation_in, "SS_InnerCryostat_bottomDome",isotope, N, mass_fv)

                elif material == 'SS_BellSideWall':
                    rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
                    data = get_xyz(rootfile)
                    bottomLip, bellWall = divide_bellWall(data)
                    calculate_rate(bottomLip, "SS_BellSideWallBottomLip",isotope, N, mass_fv)
                    calculate_rate(bellWall, "SS_BellSideWall",isotope, N, mass_fv)

                else: 
                    rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
                    data = get_xyz(rootfile)
                    calculate_rate(data, material,isotope,N, mass_fv)

#print_results_big_table()

^material ^ U238 ^ Co60 ^ K40 ^ Cs137 ^ Th228 ^ U235 ^ Th232


,,xp,yp,zp_uc,epri,ns,X,Y,Z_uc,Ed,rp,r2p,R,R2,Z,zp,typepri
entry,subentry,,,,,,,,,,,,,,,,
0,0,-662.025940,-112.835526,53.391956,0.0,1,-171.781967,-433.526611,-22.793161,224.582291,671.572937,451010.21875,466.320038,217454.37500,721.206848,797.391968,Co60
1,0,647.390625,201.387161,31.725824,0.0,1,598.018860,232.449310,-3.340017,348.170319,677.990723,459671.40625,641.606750,411659.21875,740.659973,775.725830,Co60
2,0,-286.838776,602.533325,29.241264,0.0,1,-230.820847,608.359497,-56.609818,213.223129,667.325134,445322.84375,650.676208,423379.53125,687.390198,773.241272,Co60
3,0,629.770142,-219.462158,49.572708,0.0,1,527.625916,-154.901718,-8.595600,1173.226562,666.913818,444774.03125,549.894165,302383.59375,735.404419,793.572693,Co60
4,0,428.763855,532.391296,5.366588,0.0,1,225.654221,136.078384,-82.882919,382.569550,683.578064,467278.96875,263.509308,69437.15625,661.117065,749.366577,Co60


In [75]:
def calculate_rate_pc(mat, n_gen, m_fv):
    rate_array = 0
    error_array = 0
    total_str = "|"+mat
    i = 0
    flag = 0
    for isotope in isotope_array:
        i = i+1
        if (mat == "OuterCryostat_flanges") | (mat == "OuterCryostatShell") | (mat == "OuterCryostat_Elongation"):
            material = "SS_OuterCryostat"
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data = get_xyz(rootfile)
            do(mat, data, isotope)
            
            
            
            
calculate_rate_pc("SS_OuterCryostat",N, 40000)

In [78]:
n_gen = 100000
m_fv = 4000
do("OuterCryostat_flanges", data, "Co60")

UnboundLocalError: local variable 'rate_array' referenced before assignment

In [36]:
def print_results_per_component():    
    for isotope in isotope_array:
        print("^material ^", isotope_array[0], "^", isotope_array[1], "^", isotope_array[2], "^",isotope_array[3], "^", isotope_array[4],"^",
             isotope_array[5], "^",isotope_array[6], "^")
        
        for material in materials:    
            if material == "Top_PTFE_Frame":
                data = get_TopPFTE(isotope)
                mat = "Top_PTFE_Frame"
                second_step(data, mat, isotote)
                rate_, error_rate_ = calculate_rate(data,mat,isotope, N, 4000)
                
            elif material == "Electrodes":
                data = get_electrodes(isotope)
                mat = "SS_Electrodes"
                rate_, error_rate_ = calculate_rate_pc(data, material, N, 4000)
 
            elif material == 'SS_OuterCryostat':
                rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
                data = get_xyz(rootfile)
                flange1, flange2, flange3, shell, elongation = divide_outercryo(data)
                flanges = pd.concat([flange1, flange2, flange3])
                rate1, error_rate1 = calculate_rate_pc(flanges, "OuterCryostat_flanges", N, 4000)
                rate2, error_rate2 = calculate_rate_pc(shell, "OuterCryostatShell", N, 4000)
                rate3, error_rate3 = calculate_rate_pc(elongation, "OuterCryostat_Elongation", N, 4000)           

            elif material == 'SS_InnerCryostat':
                rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
                data = get_xyz(rootfile)
                flange1_in, flange2_in, flange3_in, shell_in, elongation_in = divide_innercryo(data)
                flange_in = pd.concat([flange1_in, flange2_in, flange3_in])
                rate1, error_rate1 = calculate_rate_pc(flange_in, "SS_InnerCryostatFlanges",N, mass_fv)
                #rate2, error_rate2 = calculate_rate(shell_in, "SS_InnerCryostatShell",isotope,N, mass_fv)
                #rate3, error_rate3 = calculate_rate(elongation_in, "SS_InnerCryostat_bottomDome",isotope, N, mass_fv)

                rate1_tot = 0
                rate1_tot = rate1_tot + rate1

            elif material == 'SS_BellSideWall':
                rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
                data = get_xyz(rootfile)
                bottomLip, bellWall = divide_bellWall(data)
                rate_, error_rate_ = calculate_rate_pc(bottomLip, "SS_BellSideWallBottomLip", N, mass_fv)                
                rate1, error_rate1 = calculate_rate_pc(bellWall, "SS_BellSideWall", N, mass_fv)

            elif material == 'Teflon_TPC':
                rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
                data = get_xyz(rootfile)
                rate, error_rate = calculate_rate_TeflonTPC_pc(data, material, N, mass_fv)

            else: 
                rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
                data = get_xyz(rootfile)
                rate_, error_rate_ = calculate_rate_pc(data, material,N, mass_fv)

print_results_per_component()

^material ^ U238 ^ Co60 ^ K40 ^ Cs137 ^ Th228 ^ U235 ^ Th232 ^
0.0005209555359451852 6.353620792642709e-05
|(2.2f +- 2.2f)


UnboundLocalError: local variable 'str_' referenced before assignment

In [ ]:
DATE = 'XENONnT_2019-11-12'
materials = ["SS_OuterCryostat",
    ]
for material in materials:
    bins = 300
    min = 0
    max = 2500
    
    if material == "Top_PTFE_Frame":
        data_topPTFE = get_TopPFTE()
        material = "Top_PTFE_Frame"
        
            
        plt.hist(data_topPTFE.Ed, bins = 300, range = (0, 350), histtype = 'step', label = material)
            
    elif material == "Electrodes":
            data_electrodes = get_electrodes()
            material = "SS_Electrodes"
            plt.hist(data_electrodes.Ed,bins = bins, range = (min, max), histtype = 'step', label = material)
            
    elif material == 'SS_OuterCryostat':
        for isotope in isotope_array:
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data = get_xyz(rootfile)
            flange1, flange2, flange3, shell, elongation = divide_outercryo(data)
            flange = pd.DataFrame()
            flanges = pd.concat([flange1, flange2, flange3])
        
            
            #plt.hist(flanges.Ed, bins = bins, range = (min, max), histtype = 'step', label = "flanges")
            #plt.hist(shell.Ed, bins = bins, range = (min, max), histtype = 'step', label = "shell")
            #plt.hist(elongation.Ed, bins = bins, range = (min, max), histtype = 'step', label = "elongation")
            
    elif material == 'SS_InnerCryostat':
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data = get_xyz(rootfile)
            flange1_in, flange2_in, flange3_in, shell_in, elongation_in = divide_innercryo(data)
            flange_in = pd.concat([flange1_in, flange2_in, flange3_in])
            #plt.hist(flange_in.Ed, bins = bins, range = (min, max), histtype = 'step', label = "flange_in")
            #plt.hist(shell_in.Ed, bins = bins, range = (min, max), histtype = 'step', label = "shell_in")
            #plt.hist(elongation_in.Ed, bins = bins, range = (min, max), histtype = 'step', label = "elongation_in")
            #plt.ylabel('counts')
            #plt.xlabel('Ed [keV]')
            #plt.yscale('log')
            #plt.tight_layout()
            
    elif material == 'SS_BellSideWall':
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data = get_xyz(rootfile)
            bottomLip, bellWall = divide_bellWall(data)

    elif material == 'Teflon_TPC':
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data_teflonTPC = get_xyz(rootfile)
            

    else:
        data_per_material = pd.DataFrame()
        for isotope in isotope_array:
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data = get_xyz(rootfile)
            plt.hist(flanges.Ed, bins = bins, range = (min, max), histtype = 'step', label = material)
            data_per_material = pd.contact([data_per_material, data])
        
            

plt.hist(flanges.Ed, bins = bins, range = (min, max), histtype = 'step', label = "flanges")
plt.hist(shell.Ed, bins = bins, range = (min, max), histtype = 'step', label = "shell")
plt.hist(elongation.Ed, bins = bins, range = (min, max), histtype = 'step', label = "elongation")
#plt.hist(flange_in.Ed, bins = bins, range = (min, max), histtype = 'step', label = "flange_in")
#plt.hist(shell_in.Ed, bins = bins, range = (min, max), histtype = 'step', label = "shell_in")
#plt.hist(elongation_in.Ed, bins = bins, range = (min, max), histtype = 'step', label = "elongation_in")
plt.ylabel('counts')
plt.xlabel('Ed [keV]')
#plt.yscale('log')
plt.tight_layout()
plt.legend()


In [ ]:
print(len(shell))